# Algoritma *Naive Bayes*

## Mari berkenalan dengan Algoritma Naive Bayes

insert penjelasan here...

## Mengimpor Pustaka

Pada bagian ini akan diimpor beberapa pustaka yang akan digunakan untuk menyimulasikan Algoritma *K-Nearest Neighbor*.

In [65]:
import pandas as pd
import numpy as np

Selain itu, akan dipanggil juga modul KNN yang telah dibangun *from scratch*

In [66]:
from algorithm.naiveBayes import NaiveBayes

## Mengimpor *Dataset*

Pada bagian ini akan diimpor *dataset* yang sebelumnya telah terbagi menjadi `data_train.csv` dan `data_validation.csv`.

In [67]:
# Mengambil data train dan data validation
df_train = pd.read_csv("../data/data_train.csv")
df_validation = pd.read_csv("../data/data_validation.csv")

# *Pre-processing* Data

Tahap yang dilakukan meliputi pemisahan kolom target hingga melakukan standarisasi terhadap data sebelum dilakukan pemrosesan dengan Algoritma KNN.

In [68]:
# Melakukan pemisahan kolom target
# Menghapus baris dengan nilai yang tidak valid
df_train = df_train[df_train.px_width != 0]
df_train = df_train[df_train.px_height != 0]

# Pada bagian ini, dipilih fitur dengan nilai korelasi diatas 0.1 melalui hasil EDA
columns_to_exclude = ["blue", "wifi", "three_g", "int_memory", "sc_w", "clock_speed", "sc_h", "talk_time", "m_dep", "four_g", "n_cores", "fc", "pc", "dual_sim", "touch_screen", "mobile_wt", "price_range"]
x_train = df_train.drop(columns=columns_to_exclude)
y_train = df_train["price_range"]

x_test = df_validation.drop(columns=columns_to_exclude)
y_test = df_validation["price_range"]

In [69]:
# Dilakukan standarisasi dengan scaler yang dibuat mandiri
from utils.scaler import Scaler

scaler = Scaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

## Hasil Pemrosesan Algoritma Naive Bayes yang Dibangun

Berikut adalah hasil pemrosesan Algoritma Naive Bayes yang dibangun *from scratch*.

In [70]:
# Gunakan model KNN yang sebelumnya dibangun
nb_scratch = NaiveBayes() 

# Lakukan fit model
nb_scratch.fit(x_train, y_train)

# Lakukan prediksi dengan data validation
y_pred_scratch = nb_scratch.predict(x_test, 1e-9)

In [71]:
# Pengujian kualitas model dengan metrik
from sklearn.metrics import accuracy_score, classification_report

print(classification_report(y_test, y_pred_scratch))
print("Akurasi : ", 100 * np.round(accuracy_score(y_test, y_pred_scratch), 5), "%")

              precision    recall  f1-score   support

           0       0.88      0.89      0.88       142
           1       0.67      0.65      0.66       144
           2       0.68      0.72      0.70       155
           3       0.91      0.88      0.89       159

    accuracy                           0.79       600
   macro avg       0.79      0.78      0.78       600
weighted avg       0.79      0.79      0.79       600

Akurasi :  78.5 %


## Hasil Pemrosesan Algoritma Naive Bayes Pembanding

Hasil pemrosesan diatas akan dibandingkan dengan hasil yang diperoleh dari *library* scikit-learn

In [72]:
# Pemanggilan model KNN dari scikit-learn
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KernelDensity
from sklearn.pipeline import make_pipeline

gnb = GaussianNB()
y_pred = gnb.fit(x_train, y_train)
y_pred_scikit = gnb.predict(x_test)


In [73]:
# Pengujian kualitas model dengan metrik
print(classification_report(y_test, y_pred_scikit))
print("Akurasi : ", 100 * np.round(accuracy_score(y_test, y_pred_scikit), 5), "%")

              precision    recall  f1-score   support

           0       0.88      0.89      0.88       142
           1       0.67      0.65      0.66       144
           2       0.68      0.72      0.70       155
           3       0.91      0.88      0.89       159

    accuracy                           0.79       600
   macro avg       0.79      0.78      0.78       600
weighted avg       0.79      0.79      0.79       600

Akurasi :  78.5 %


## Hasil yang diperoleh

Berdasarkan kedua hasil tersebut, diperoleh **hasil yang sama** antara nilai prediksi yang dihasilkan oleh model Naive Bayes yang dibangun *from scratch* dengan model yang dimiliki scikit-learn.

In [74]:
# Pengujian kualitas model dengan metrik
print(classification_report(y_pred_scratch, y_pred_scikit))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       143
           1       1.00      1.00      1.00       140
           2       1.00      1.00      1.00       163
           3       1.00      1.00      1.00       154

    accuracy                           1.00       600
   macro avg       1.00      1.00      1.00       600
weighted avg       1.00      1.00      1.00       600



## Apakah sudah merupakan model yang terbaik?

OTW diubah

In [75]:
# Menggunakan grid search, cari nilai variable smoothing terbaik

from sklearn.model_selection import GridSearchCV

param_grid_nb = {
    'var_smoothing': np.logspace(0,-9, num=400)
}
bestNB = GridSearchCV(estimator=GaussianNB(), param_grid=param_grid_nb, verbose=1, cv=10, n_jobs=-1)
bestNB.fit(x_train, y_train)
print(bestNB.best_estimator_)
best_param = bestNB.best_params_
y_pred_scikit = bestNB.best_estimator_.predict(x_test)

Fitting 10 folds for each of 400 candidates, totalling 4000 fits


GaussianNB(var_smoothing=0.5647805074067556)


In [76]:
# Pengujian kualitas model dengan metrik
print(classification_report(y_test, y_pred_scikit))
print("Akurasi : ", 100 * np.round(accuracy_score(y_test, y_pred_scikit), 5), "%")

              precision    recall  f1-score   support

           0       0.93      0.99      0.96       142
           1       0.75      0.69      0.72       144
           2       0.72      0.75      0.73       155
           3       0.96      0.92      0.94       159

    accuracy                           0.84       600
   macro avg       0.84      0.84      0.84       600
weighted avg       0.84      0.84      0.84       600

Akurasi :  84.0 %


In [77]:
# Gunakan model KNN yang sebelumnya dibangun
nb_scratch = NaiveBayes() 

# Lakukan fit model
nb_scratch.fit(x_train, y_train)

# Lakukan prediksi dengan data validation, menggunakan var_smoothing terbaik yang ditemukan
y_pred_scratch = nb_scratch.predict(x_test, 0.5647805074067556)

In [78]:
# Pengujian kualitas model dengan metrik
from sklearn.metrics import accuracy_score, classification_report

print(classification_report(y_test, y_pred_scratch))
print("Akurasi : ", 100 * np.round(accuracy_score(y_test, y_pred_scratch), 5), "%")

              precision    recall  f1-score   support

           0       0.93      0.99      0.96       142
           1       0.73      0.69      0.71       144
           2       0.71      0.74      0.72       155
           3       0.96      0.91      0.94       159

    accuracy                           0.83       600
   macro avg       0.83      0.83      0.83       600
weighted avg       0.83      0.83      0.83       600

Akurasi :  83.333 %


## Penyimpanan dan *Load* Model

Agar model dapat digunakan kembali, maka model harus dapat disimpan dan di-*load*. Berikut adalah implementasi penyimpanan model yang dilakukan menggunakan *library* pickle.

In [79]:
import pickle

# Menyimpan model dalam pkl
model_pkl_file = "models/naive_bayes_model.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(nb_scratch, file)

Untuk membuktikan bahwa model berhasil tersimpan, berikut adalah pembuktian pemanggilan kembali hasil prediksi model yang disimpan

In [80]:
# Load kembali model dari pkl
with open(model_pkl_file, 'rb') as file:  
    model = pickle.load(file)

# Melakukan prediksi dengan model tersebut
y_pickle = model.predict(x_test, 0.5647805074067556)

# Menguji hasil akurasi model
print(classification_report(y_test, y_pickle)) 

              precision    recall  f1-score   support

           0       0.93      0.99      0.96       142
           1       0.73      0.69      0.71       144
           2       0.71      0.74      0.72       155
           3       0.96      0.91      0.94       159

    accuracy                           0.83       600
   macro avg       0.83      0.83      0.83       600
weighted avg       0.83      0.83      0.83       600



Terbukti bahwa model pkl yang disimpan berhasil untuk digunakan kembali.

## [Bonus] Submisi Kaggle

Bagian ini dikhususkan untuk pemrosesan data dan penggunaan model yang dibuat sebagai dasar membuat submisi pada Kaggle.

In [81]:
# Impor dataset
test_data = pd.read_csv('../data/test.csv')

columns_to_exclude_new = ["id", "blue", "wifi", "three_g", "int_memory", "sc_w", "clock_speed", "sc_h", "talk_time", "m_dep", "four_g", "n_cores", "fc", "pc", "dual_sim", "touch_screen", "mobile_wt"]
features = test_data.drop(columns=columns_to_exclude_new)

# Gunakan model KNN yang sebelumnya dibangun
nb_kaggle = NaiveBayes()

# Lakukan fit model
nb_kaggle.fit(x_train, y_train)

# Lakukan prediksi dengan data validation
feature_test = scaler.transform(features)
predictions = nb_kaggle.predict(feature_test, 0.5647805074067556)

# Membuat dataframe hasil
result_df = pd.DataFrame({'id': test_data['id'], 'price_range': predictions})

# Menyimpan dataframe dalam csv
result_df.to_csv('../result/predictions-naive-bayes.csv', index=False)